# Tutorial for ELASTIC

In [ ]:
import os, sys

sys.path.append('../mcunet')
sys.path.append('../once-for-all')

!pwd
!ls
print()

sys.path

In [ ]:
import torch, numpy as np, random
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
import pandas as pd, ast
from torch import nn, optim

from ofa.nas.accuracy_predictor.acc_predictor import AccuracyPredictor
from search import IterativeEvolutionFinder
from utils import MobileDetArchEncoder, SizePredictor

## Train Accuracy Predictor

In [ ]:
# prepare data
files = [f"outputk5e{i}.xlsx" for i in range(6, 0, -1)] + [f"outputk3e{i}.xlsx" for i in range(6, 0, -1)] + [f"outputk1e{i}.xlsx" for i in range(6, 0, -1)]

x, y = [], []
for file in files:
    df = pd.read_excel(file)
    subnet = df['subnet'].apply(ast.literal_eval).tolist()
    acc = df['acc']
    values = []
    for s in acc:
        t = eval(s, {"np": np})
        values.append(float(t[2]))
    x.extend(subnet)
    y.extend(values)

idx = torch.randperm(len(x))
x = [x[i] for i in idx]
y = np.array(y)[idx]

encoder = MobileDetArchEncoder(
    ks_list=[1, 3, 5],
    expand_list=[1, 2, 3, 4, 5, 6],
    depth_list=[1, 2, 3, 4, 5, 6],
    n_stage=4
)

x = torch.stack([
    torch.tensor(encoder.arch2feature({**arch, 'image_size': 224}), dtype=torch.float32)
    for arch in x
])
y = torch.tensor(y, dtype=torch.float32)

factor = 0.8
x_train = x[:int(len(x)*factor)]
y_train = y[:int(len(y)*factor)]
x_test = x[int(len(x)*factor):]
y_test = y[int(len(y)*factor):]

In [ ]:
# train accuracy predictor
predictor = AccuracyPredictor(
    encoder,
    checkpoint_path=None,
    device='cpu'
)
criterion = nn.MSELoss()
optimizer = optim.SGD(predictor.parameters(), lr=0.01)

for epoch in range(10000):
    predictor.train()
    y_pred = predictor(x_train)
    loss = criterion(y_pred, y_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        predictor.eval()
        with torch.no_grad():
            y_pred = predictor(x_test)
            acc = criterion(y_pred, y_test)
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")
        print(f"Test Loss: {acc.item():.4f}")

torch.save(predictor.state_dict(), 'predictor.pt')

## Iterative Search

In [ ]:
# conduct iterative search
finder = IterativeEvolutionFinder(
    efficiency_predictor=SizePredictor(
        n_classes=20,
        bn_param=(0.1, 1e-3),
        width_mult_list=1.5,
        ks_list=[1, 3, 5],
        expand_ratio_list=[1, 2, 3, 4, 5, 6],
        depth_list=[1, 2, 3, 4, 5],
    ),
    accuracy_predictor=AccuracyPredictor(
        MobileDetArchEncoder(
            ks_list=[1, 3, 5],
            expand_list=[1, 2, 3, 4, 5, 6],
            depth_list=[1, 2, 3, 4, 5],
            n_stage=4
        ),
        checkpoint_path='predictor.pt',
        device='cpu'
    ),
    arch_mutate_prob=0.1,
    resolution_mutate_prob=0.5,
    population_size=100,
    max_time_budget=100 // 10,
    parent_ratio=0.25,
    mutation_ratio=0.5,
    passthrough=0.5,
)

constraint = {
    'minimum_weights': 1000000,
    'maximum_weights': 1200000,
}
info = finder.run_evolution_search(constraint, 50, True)
info